Initialization

In [ ]:
!apt-get update install -y libenchant-2-dev --fix-missing
!pip install pyenchant -v
!pip install Levenshtein
!pip install spacy
!pip install PyEnchant
!pip install homoglyphs
!pip install textattack
!pip install -q kaggle
!pip install tensorflow_text tensorflow_hub
!pip install tensorflow
!pip install evaluate
!pip install unidecode

!git config --global --unset https.proxy
!git clone https://github.com/lethaiq/perturbations-in-the-wild

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common enchant-2 hunspell-en-us libaspell15 libblkid-dev libblkid1
  libenchant-2-2 libffi-dev libglib2.0-dev libglib2.0-dev-bin libhunspell-1.7-0 libmount-dev
  libmount1 libselinux1-dev libsepol-dev libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell | openoffice.org-core
  libenchant-2-voikko libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils cryptsetup-bin
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common enchant-2 hunspell-en-us libaspell15 libblkid-dev
  libenchant-2-2 libenchant-2-dev libffi-dev libglib2.0-dev libglib2.0-dev-bin libhunspell-1.7-0
  libmount-dev libselinux1-dev libsepol-dev libtext-iconv-perl
The following packages will be upgraded:
  libblkid1 libmoun

In [ ]:
import torch
import transformers
from scipy.special import softmax
import numpy as np
import pandas as pd
from google.colab import files
import json
import tensorflow as tf
import tensorflow_text as text
#import tensorflow_hub as hub
import datasets
import enchant
from unidecode import unidecode


In [ ]:
text = "Björn, Łukasz and Σωκράτης."
print(unidecode(text))
text = "𝔅𝔧ö𝔯𝔫, Ł𝔲𝔨𝔞𝔰𝔷 𝔞𝔫𝔡 Σωκράτης."
print(unidecode(text))
text = "🅱🎷ö🌱🥄, Ł⛎🎉🅰💲💤 🅰🥄🐬 Σωκράτης."
print(unidecode(text))
text = "⒝⒥ö⒭⒩, Ł⒰⒦⒜⒮⒵ ⒜⒩⒟ Σωκράτης."
print(unidecode(text))
text = "𝔹𝕛ö𝕣𝕟, Ł𝕦𝕜𝕒𝕤𝕫 𝕒𝕟𝕕 Σωκράτης."
print(unidecode(text))
text = "͓̽B͓͓̽̽j͓̽ö͓̽r͓͓̽̽n͓̽, Ł͓̽u͓͓̽̽k͓̽a͓͓̽̽s͓͓̽̽z͓̽ a͓͓̽̽n͓͓̽̽d͓̽ Σωκράτης."
print(unidecode(text))

Bjorn, Lukasz and Sokrates.
Bjorn, Lukasz and Sokrates.
[B]o, L[A] [A] Sokrates.
(b)(j)o(r)(n), L(u)(k)(a)(s)(z) (a)(n)(d) Sokrates.
Bjorn, Lukasz and Sokrates.
Bjorn, Lukasz and Sokrates.


In [ ]:

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"harleygarrett","key":"579d98848fe4616e7d0389d924be8a33"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c human-vs-ai-text-classification-feb2024

!mkdir train
!unzip -o human-vs-ai-text-classification-feb2024.zip -d train

100% 306M/306M [00:09<00:00, 39.8MB/s]
100% 306M/306M [00:09<00:00, 32.3MB/s]
Archive:  human-vs-ai-text-classification-feb2024.zip
  inflating: train/sample_submission.csv  
  inflating: train/test.csv          
  inflating: train/train.csv         


In [ ]:
test = pd.read_csv("train/test.csv")
train = pd.read_csv("train/train.csv")
test.shape, train.shape

((86587, 2), (341064, 2))

In [ ]:
from sklearn.model_selection import train_test_split
train_y = train['generated'].values
train_x = train['text'].values
#train_1x, train_2x, train_1y, train_2y = train_test_split(train_x, train_y, test_size = 0.2)
train_1x, train_2x, train_1y, train_2y = train_test_split(train_x, train_y, test_size = 2500, train_size=5000)
train_1x.shape, train_2y.shape
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}

NLP Model Creation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
  tokenized_batch = tokenizer(examples["text"], truncation=True, padding=True, max_length=500)
  tokenized_batch["label"] = [label for label in examples["label"]]
  return tokenized_batch
data = datasets.DatasetDict({"train":datasets.Dataset.from_dict({'label':train_1y, 'text':train_1x}), "test":datasets.Dataset.from_dict({'label':train_2y, 'text':train_2x})})
tokenized_data = data.map(preprocess_function, batched=True, batch_size=500)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="Team10",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to = "none",
)
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.049714
2,No log,0.048053


TrainOutput(global_step=314, training_loss=0.10689509446453896, metrics={'train_runtime': 2873.4601, 'train_samples_per_second': 3.48, 'train_steps_per_second': 0.109, 'total_flos': 1293626940000000.0, 'train_loss': 0.10689509446453896, 'epoch': 2.0})

In [ ]:
trainer.save_model('./Team10BestModel')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "./Team10BestModel", num_labels=2, id2label=id2label, label2id=label2id
)

NLP Model Attacking (part 1)

NLP Model Reinforcement <br>
(Apply pipeline to prevent attacks from part 1, and train model to be more robust against attacks)

In [ ]:
%ls
%cd perturbations-in-the-wild/
%ls
from anthro_lib import ANTHRO
anthro = ANTHRO()
anthro.load('./ANTHRO_Data_V1.0')
%cd ..

human-vs-ai-text-classification-feb2024.zip  sample_data/  Team10BestModel/
perturbations-in-the-wild/                   Team10/       train/
/content/perturbations-in-the-wild
ANTHRO_Data_V1.0/  anthro_lib.py  LICENSE  logo.jpg  README.md  test.py
loading... token2idx
loading... level2code2token
loading... token2spell
loaded
UPPD VERSION Public
Total unique codes:
Level 1: 407620 codes


100%|██████████| 1385430/1385430 [00:01<00:00, 1168942.68it/s]


done
/content


In [ ]:
# def preprocess_function_adv(examples):
#   text = str(examples["text"])

#   if np.random.choice([0, 1], p=[0.5, 0.5]): #50/50 to use an attacked text
#     tokenized_batch = tokenizer(text, truncation=True, padding=True, max_length=500)
#     tokenized_batch["label"] = [label for label in examples["label"]]
#     return tokenized_batch
#   else:
#     pert_text = []
#     for token in text.split():
#       if np.random.choice([0, 1], p=[0.5, 0.5]): #50/50 to disturb a single token
#           choices = anthro.get_similars(token, level=1, distance=5, strict=True)
#           if len(choices):
#               choice = np.random.choice(list(choices))
#               pert_text.append(choice)
#           else:
#               pert_text.append(token)
#       else:
#           pert_text.append(token)
#     pert_text = " ".join(pert_text)
#     tokenized_batch = tokenizer(pert_text, truncation=True, padding=True, max_length=500)
#     tokenized_batch["label"] = [label for label in examples["label"]]
#     return tokenized_batch


adv_data_train = datasets.DatasetDict({"train":datasets.Dataset.from_dict({'label':train_1y, 'text':train_1x})})

for text in adv_data_train['train']['text']:
  if np.random.choice([0, 1], p=[0.5, 0.5]): #50/50 to use an attacked text
    #nothing
    continue
  else:
    pert_text = []
    for token in text.split():
      if np.random.choice([0, 1], p=[0.5, 0.5]): #50/50 to disturb a single token
          choices = anthro.get_similars(token, level=1, distance=5, strict=True)
          if len(choices):
              choice = np.random.choice(list(choices))
              pert_text.append(choice)
          else:
              pert_text.append(token)
      else:
          pert_text.append(token)
    pert_text = " ".join(pert_text)
    text = pert_text

adv_data_test = datasets.DatasetDict({"test":datasets.Dataset.from_dict({'label':train_2y, 'text':train_2x})})
adv_tokenized_data_train = adv_data_train.map(preprocess_function, batched=True, batch_size = 500)
adv_tokenized_data_test = adv_data_test.map(preprocess_function, batched=True, batch_size=500)
adv_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
adv_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
adv_training_args = TrainingArguments(
    output_dir="Team10Adv",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to = "none",
)
adv_trainer = Trainer(
    model=adv_model,
    args=adv_training_args,
    train_dataset=adv_tokenized_data_train["train"],
    eval_dataset=adv_tokenized_data_test["test"],
    tokenizer=tokenizer,
    data_collator=adv_data_collator
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
adv_trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.050954
2,No log,0.051829


TrainOutput(global_step=314, training_loss=0.09736273698746019, metrics={'train_runtime': 7320.2455, 'train_samples_per_second': 1.366, 'train_steps_per_second': 0.043, 'total_flos': 1293626940000000.0, 'train_loss': 0.09736273698746019, 'epoch': 2.0})

In [ ]:
trainer.save_model('./Team10BestModelAdv')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "./Team10BestModelAdv", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:

test = tokenizer("test")
preds = trainer.predict(tokenized_data["train"])
from sklearn.metrics import accuracy_score
accuracy_score(preds.label_ids, preds.predictions.argmax(1))

# test = tokenizer("", return_tensors="pt")
# output = model(**test)
# def softmax(x):
#     e_x = np.exp(x - np.max(x))
#     return e_x / e_x.sum(axis=0) # only difference
# softmax(output[0].detach().numpy()[0]), output[0].detach().numpy()[0]

0.9954

NLP Model Attacking (part 2)

Competition Templating

In [ ]:
class RedBlueModelWrapper:
  def __init__(self):
    # TODO
    # IMPORT ALL IMPORTANT LIBRARIES HERE
    # INITIALIZE AND LOAD YOUR MODEL, TOKENIZER
    # AND OTHER NECCESSARY COMPONENTS HERE IF NEEDED
    self.total_queries = 0
    self.TIME_DURATION_ATTEMPTS = 20
    self.query_time = []

  def _query(self, text: str):
    # TODO
    # PREDICTION OF YOUR MODEL HERE ON THE INPUT TEXT ``text''
    # MUST RETURN A PREDICTION PROBABILITY VECTOR IN NUMPY
    # THIS VECTOR SHOULD HAVE SHAPE (2,),e.g., array([0.79184294, 0.20815705])
    # THE FIRST COMPONENT IS THE PROBABILITY THE TEXT IS WRITTEN BY HUMAN
    # THE SECOND COMPONENT IS THE PROBABILITY THE TEXT IS GENERATED BY MACHINE
    # ALL PROBABILITIES NEED TO BE SUMED TO 1.0
    raise NotImplementedError

  def __call__(self, text: str):
    if len(self.query_time) < self.TIME_DURATION_ATTEMPTS:
      start_time = time.time()

    probs = self._query(text)
    assert probs.shape == (2,)
    assert np.abs(np.sum(probs) - 1.0) < 0.001

    if len(self.query_time) < self.TIME_DURATION_ATTEMPTS:
      end_time = time.time()
      duration = end_time - start_time
      self.query_time.append(duration)

    self.increase_query()
    return probs

  def increase_query(self):
    self.total_queries += 1

  def reset_query(self):
    self.total_queries = 0.0

  def average_time(self):
    return np.mean(self.query_time)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install Levenshtein   #uncomment out for competition
!pip install spacy
!pip install PyEnchant
!pip install homoglyphs
!pip install textattack
!pip install -q kaggle
!pip install tensorflow_text tensorflow_hub
!pip install tensorflow
!pip install evaluate
!pip install unidecode

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from unidecode import unidecode
import numpy as np
import time
class RedBlueModelWrapper_Team10(RedBlueModelWrapper):
  def __init__(self):
    super(RedBlueModelWrapper_Team10, self).__init__() # THIS LINE IS IMPORTANT AND MUST BE PLACED HERE
    id2label = {0: "human", 1: "machine"}
    label2id = {"human": 0, "machine": 1}
    !mkdir Team10BestModelAdv
    !unzip -o Team10BestModelAdv.zip
    self.model = AutoModelForSequenceClassification.from_pretrained(
    "./Team10BestModelAdv", num_labels=2, id2label=id2label, label2id=label2id
)
    self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    # INITIALIZE AND LOAD YOUR MODEL, TOKENIZER
    # AND OTHER NECCESSARY COMPONENTS HERE IF NEEDED
    self.total_queries = 0
    self.TIME_DURATION_ATTEMPTS = 20
    self.query_time = []

  def softmax(self, x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
  def _query(self, text: str):
    test = self.tokenizer(unidecode(text), return_tensors="pt")
    output = self.model(**test)
    return self.softmax(output[0].detach().numpy()[0])

In [ ]:
test = RedBlueModelWrapper_Team10()

mkdir: cannot create directory ‘Team10BestModelAdv’: File exists
Archive:  Team10BestModelAdv.zip
  inflating: Team10BestModelAdv/config.json  
  inflating: Team10BestModelAdv/model.safetensors  
  inflating: Team10BestModelAdv/special_tokens_map.json  
  inflating: Team10BestModelAdv/tokenizer.json  
  inflating: Team10BestModelAdv/tokenizer_config.json  
  inflating: Team10BestModelAdv/training_args.bin  
  inflating: Team10BestModelAdv/vocab.txt  


In [ ]:
import time

test._query("test!"), test.__call__("test!")

(array([0.5131935, 0.4868065], dtype=float32),
 array([0.5131935, 0.4868065], dtype=float32))